# Converters for Quadratic Programs

Optimization problems in Qiskit optimization module are represented with the `QuadraticProgram` class, which is a generic and powerful representation for optimization problems. In general, optimization algorithms are defined for a certain formulation of a quadratic program, and we need to convert our problem to the right type.

For instance, Qiskit optimization provides several optimization algorithms that can handle [Quadratic Unconstrained Binary Optimization](https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization) (QUBO) problems. These are mapped to Ising Hamiltonians, for which Qiskit optimization uses the `qiskit.quantum_info.SparsePauliOp` object, and then their ground state is approximated. For this optimization, commonly known algorithms such as VQE or QAOA can be used as underlying routine. See the following tutorial about the [Minimum Eigen Optimizer](./03_minimum_eigen_optimizer.ipynb) for more detail. Note that also other algorithms exist that work differently, such as the `GroverOptimizer`.

To map a problem to the correct input format, the optimization module of Qiskit optimization offers a variety of converters. In this tutorial we're providing an overview on this functionality. Currently, Qiskit optimization contains the following converters.

- `InequalityToEquality`: convert inequality constraints into equality constraints with additional slack variables.
- `IntegerToBinary`: convert integer variables into binary variables and corresponding coefficients.
- `LinearEqualityToPenalty`: convert equality constraints into additional terms of the objective function.
- `LinearInequalityToPenalty`: convert inequality constraints into additional terms of the objective function.
- `MaximizeToMinimize`: convert to the equivalent minimization problem.
- `MinimizeToMaximize`: convert to the equivalent maximization problem.
- `QuadraticProgramToQubo`: a wrapper that includes `InequalityToEquality`, `IntegerToBinary`, `LinearEqualityToPenalty`, `LinearInequalityToPenalty`, and `MaximizeToMinimize` for convenience.

## InequalityToEquality
`InequalityToEqualityConverter` converts inequality constraints into equality constraints with additional slack variables to remove inequality constraints from `QuadraticProgram`. The upper bounds and the lower bounds of slack variables will be calculated from the difference between the left sides and the right sides of constraints. Signs of slack variables depend on symbols in constraints such as $\leq$ and $\geq$.

The following is an example of a maximization problem with two inequality constraints. Variable $x$ and $y$ are binary variables and variable $z$ is an integer variable.

\begin{aligned}
   & \text{maximize}
       & 3x + 2y + z\\
   & \text{subject to:}
       & x+y+z \leq 5.5\\
       & & x+y+z \geq 2.5\\
       & & x, y \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5,6,7\} \\
\end{aligned}

With `QuadraticProgram`, an optimization model of the problem is written as follows.

In [1]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators.docplex_mp import to_docplex_mp

In [2]:
qp = QuadraticProgram()
qp.binary_var("x")
qp.binary_var("y")
qp.integer_var(lowerbound=0, upperbound=7, name="z")

qp.maximize(linear={"x": 3, "y": 2, "z": 1})
qp.linear_constraint(linear={"x": 1, "y": 1, "z": 1}, sense="LE", rhs=5.5, name="xyz_leq")
qp.linear_constraint(linear={"x": 1, "y": 1, "z": 1}, sense="GE", rhs=2.5, name="xyz_geq")
print(qp.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z

Subject to
  Linear constraints (2)
    x + y + z <= 5.5  'xyz_leq'
    x + y + z >= 2.5  'xyz_geq'

  Integer variables (1)
    0 <= z <= 7

  Binary variables (2)
    x y



Call `convert` method of `InequalityToEquality` to convert.

In [3]:
from qiskit_optimization.converters import InequalityToEquality

In [4]:
ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(qp)
print(qp_eq.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack + y + z == 5  'xyz_leq'
    x - xyz_geq@int_slack + y + z == 3  'xyz_geq'

  Integer variables (3)
    0 <= z <= 7
    0 <= xyz_leq@int_slack <= 5
    0 <= xyz_geq@int_slack <= 6

  Binary variables (2)
    x y



After converting, the formulation of the problem looks like the above output. As we can see, the inequality constraints are replaced with equality constraints with additional integer slack variables, $xyz\_leg\text{@}int\_slack$ and $xyz\_geq\text{@}int\_slack$. 

Let us explain how the conversion works. For example, the lower bound of the left side of the first constraint is $0$ which is the case of $x=0$, $y=0$, and $z=0$. Thus, the upper bound of the additional integer variable must be $5$ to be able to satisfy even the case of $x=0$, $y=0$, and $z=0$. Note that we cut off the part after the decimal point in the converted formulation since the left side of the first constraint in the original formulation can be only integer values. For the second constraint, basically we apply the same approach. However, the symbol in the second constraint is $\geq$, so we add minus before $xyz\_geq\text{@}int\_slack$ to be able to satisfy even the case of $x=1, y=1$, and $z=7$.

\begin{aligned}
   & \text{maximize}
       & 3x + 2y + z\\
   & \text{subject to:}
       & x+y+z+ xyz\_leg\text{@}int\_slack= 5\\
       & & x+y+z+xyz\_geq\text{@}int\_slack= 3\\
       & & x, y \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5,6,7\} \\
       & & xyz\_leg\text{@}int\_slack \in \{0,1,2,3,4,5\} \\
       & & xyz\_geq\text{@}int\_slack \in \{0,1,2,3,4,5,6\} \\
\end{aligned}

Let us see how the `interpret` method works. The purpose of this method is to convert the solution of the converted problem back to that of the original problem. To use this method, we would first require to solve the problem. For the purpose of this tutorial, we will use docplex to solve. We will first translate the quadratic problem into a docplex.mp model.

In [5]:
from qiskit_optimization.algorithms import CplexOptimizer

cplex_optimizer = CplexOptimizer()

In [6]:
result_orig = cplex_optimizer.solve(qp)
print(result_orig)

fval=8.0, x=1.0, y=1.0, z=3.0, status=SUCCESS


In [7]:
result_eq = cplex_optimizer.solve(qp_eq)
print(result_eq)

fval=8.0, x=1.0, y=1.0, z=3.0, xyz_leq@int_slack=0.0, xyz_geq@int_slack=2.0, status=SUCCESS


The result `result_eq` of `qp_eq` has 5 variable values (`x=1.0, y=1.0, z=3.0, xyz_leq@int_slack=0.0, xyz_geq@int_slack=2.0`) while result `result_orig` of the original `qp` has three values (`x=1.0, y=1.0, z=3.0`).
We can call `InequalityToEquality.interpret` method by passing a list or an array to the method that has values of `qp_eq.variables` as follows.
`result_eq.x` has the list of values that each variable takes in the solution in correspondence to their position in the variable list `qp_eq.variables`.

In [8]:
print("interpreting values of result_eq:", ineq2eq.interpret(result_eq.x))
print("values of result_orig:", result_orig.x)

interpreting values of result_eq: [1. 1. 3.]
values of result_orig: [1. 1. 3.]


We notice that $[1., 1., 3.]$ are the values taken in the solution of the converted problem for the common variables between converted and original problems (variables: $x$, $y$, $z$). The interpret method shows the same values are the solution of the original problem. This is because the objective function for the converted and original problems is exactly the same. The slack variables are just ensuring equality in the constraints of the converted problem, where the constraints are also exactly same between the original and converted problems, except that the original problem has inequality constraints.

## IntegerToBinary

`IntegerToBinary` converts integer variables into binary variables and coefficients to remove integer variables from `QuadraticProgram`. For converting, bounded-coefficient encoding proposed in [arxiv:1706.01945](https://arxiv.org/abs/1706.01945) (Eq. (5)) is used. For more detail of the encoding method, please see the paper.

We use the output of `InequalityToEquality` as a starting point. Variables $x$ and $y$ are binary variables, while the variable $z$ and the slack variables $xyz\_leq\text{@}int\_slack$ and $xyz\_geq\text{@}int\_slack$ are integer variables. We print the problem again for reference.

In [9]:
print(qp_eq.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack + y + z == 5  'xyz_leq'
    x - xyz_geq@int_slack + y + z == 3  'xyz_geq'

  Integer variables (3)
    0 <= z <= 7
    0 <= xyz_leq@int_slack <= 5
    0 <= xyz_geq@int_slack <= 6

  Binary variables (2)
    x y



Call `convert` method of `IntegerToBinary` to convert.

In [10]:
from qiskit_optimization.converters import IntegerToBinary

In [11]:
int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
print(qp_eq_bin.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z@0 + 2*z@1 + 4*z@2

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack@0 + 2*xyz_leq@int_slack@1 + 2*xyz_leq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 5  'xyz_leq'
    x - xyz_geq@int_slack@0 - 2*xyz_geq@int_slack@1 - 3*xyz_geq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 3  'xyz_geq'

  Binary variables (11)
    x y z@0 z@1 z@2 xyz_leq@int_slack@0 xyz_leq@int_slack@1 xyz_leq@int_slack@2
    xyz_geq@int_slack@0 xyz_geq@int_slack@1 xyz_geq@int_slack@2



After converting, the integer variable $z$ is replaced with three binary variables $z\text{@}0$, $z\text{@}1$ and $z\text{@}2$ with coefficients 1, 2 and 4, respectively as the above. 
The slack variables $xyz\_leq\text{@}int\_slack$ and $xyz\_geq\text{@}int\_slack$ that were introduced by `InequalityToEquality` are also both replaced with three binary variables with coefficients 1, 2, 2, and  1, 2, 3, respectively.

Note: Essentially the coefficients mean that the sum of these binary variables with coefficients can be the sum of a subset of $\{1, 2, 4\}$, $\{1, 2, 2\}$, and $\{1, 2, 3\}$ to represent that acceptable values $\{0, \ldots, 7\}$, $\{0, \ldots, 5\}$, and $\{0, \ldots, 6\}$, which respects the lower bound and the upper bound of original integer variables correctly.

`IntegerToBinary` also provides `interpret` method that is the functionality to translate a given binary result back to the original integer representation. Let us see how the `interpret` method works.

In [12]:
result_eq = cplex_optimizer.solve(qp_eq)
print(result_eq)

fval=8.0, x=1.0, y=1.0, z=3.0, xyz_leq@int_slack=0.0, xyz_geq@int_slack=2.0, status=SUCCESS


In [13]:
result_eq_bin = cplex_optimizer.solve(qp_eq_bin)
print(result_eq_bin)

fval=8.0, x=1.0, y=1.0, z@0=1.0, z@1=1.0, z@2=0.0, xyz_leq@int_slack@0=0.0, xyz_leq@int_slack@1=0.0, xyz_leq@int_slack@2=0.0, xyz_geq@int_slack@0=0.0, xyz_geq@int_slack@1=1.0, xyz_geq@int_slack@2=0.0, status=SUCCESS


`result_eq_bin` has more binary variables due to the conversion by `IntegerToBinary.convert`.
`IntegerToBinary.interpret` translates them back to the integer values by aggregating binary variables values associated with the original integer variables of `qp_eq`.

In [14]:
print("interpreting values of result_eq_bin:", int2bin.interpret(result_eq_bin.x))
print("values of result_eq:", result_eq.x)

interpreting values of result_eq_bin: [1. 1. 3. 0. 2.]
values of result_eq: [1. 1. 3. 0. 2.]


## LinearEqualityToPenalty

`LinearEqualityToPenalty` converts linear equality constraints into additional quadratic penalty terms of the objective function to map `QuadraticProgram` to an unconstrained form.
An input to the converter has to be a `QuadraticProgram` with only linear equality constraints. Those equality constraints, e.g. $\sum_i a_i x_i  = b$ where $a_i$ and $b$ are numbers and $x_i$ is a variable, will be added to the objective function in the form of $M(b - \sum_i a_i x_i)^2$ where $M$ is a large number as penalty factor. 
By default $M= 1e5$. The sign of the term depends on whether the problem type is a maximization or minimization.

We use the output of `IntegerToBinary` as a starting point, where all variables are binary variables and all inequality constraints have been mapped to equality constraints. 
We print the problem again for reference.

In [15]:
print(qp_eq_bin.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z@0 + 2*z@1 + 4*z@2

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack@0 + 2*xyz_leq@int_slack@1 + 2*xyz_leq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 5  'xyz_leq'
    x - xyz_geq@int_slack@0 - 2*xyz_geq@int_slack@1 - 3*xyz_geq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 3  'xyz_geq'

  Binary variables (11)
    x y z@0 z@1 z@2 xyz_leq@int_slack@0 xyz_leq@int_slack@1 xyz_leq@int_slack@2
    xyz_geq@int_slack@0 xyz_geq@int_slack@1 xyz_geq@int_slack@2



Call `convert` method of `LinearEqualityToPenalty` to convert.

In [16]:
from qiskit_optimization.converters import LinearEqualityToPenalty

In [17]:
lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qp_eq_bin)
print(qubo.prettyprint())

Problem name: 

Maximize
  -26*x^2 + 26*x*xyz_geq@int_slack@0 + 52*x*xyz_geq@int_slack@1
  + 78*x*xyz_geq@int_slack@2 - 26*x*xyz_leq@int_slack@0
  - 52*x*xyz_leq@int_slack@1 - 52*x*xyz_leq@int_slack@2 - 52*x*y - 52*x*z@0
  - 104*x*z@1 - 208*x*z@2 - 13*xyz_geq@int_slack@0^2
  - 52*xyz_geq@int_slack@0*xyz_geq@int_slack@1
  - 78*xyz_geq@int_slack@0*xyz_geq@int_slack@2 - 52*xyz_geq@int_slack@1^2
  - 156*xyz_geq@int_slack@1*xyz_geq@int_slack@2 - 117*xyz_geq@int_slack@2^2
  - 13*xyz_leq@int_slack@0^2 - 52*xyz_leq@int_slack@0*xyz_leq@int_slack@1
  - 52*xyz_leq@int_slack@0*xyz_leq@int_slack@2 - 52*xyz_leq@int_slack@1^2
  - 104*xyz_leq@int_slack@1*xyz_leq@int_slack@2 - 52*xyz_leq@int_slack@2^2
  + 26*y*xyz_geq@int_slack@0 + 52*y*xyz_geq@int_slack@1
  + 78*y*xyz_geq@int_slack@2 - 26*y*xyz_leq@int_slack@0
  - 52*y*xyz_leq@int_slack@1 - 52*y*xyz_leq@int_slack@2 - 26*y^2 - 52*y*z@0
  - 104*y*z@1 - 208*y*z@2 + 26*z@0*xyz_geq@int_slack@0
  + 52*z@0*xyz_geq@int_slack@1 + 78*z@0*xyz_geq@int_slack@2
  -

After converting, the equality constraints are added to the objective function as additional terms with the default penalty factor provided by Qiskit optimization.
The resulting problem is now a QUBO and compatible with many quantum optimization algorithms such as VQE, QAOA and so on.

This gives the same result as before.

Like we did for the other converters, let us see how `interpret` method works for this case. 

In [18]:
result_eq_bin = cplex_optimizer.solve(qp_eq_bin)
print(result_eq_bin)

fval=8.0, x=1.0, y=1.0, z@0=1.0, z@1=1.0, z@2=0.0, xyz_leq@int_slack@0=0.0, xyz_leq@int_slack@1=0.0, xyz_leq@int_slack@2=0.0, xyz_geq@int_slack@0=0.0, xyz_geq@int_slack@1=1.0, xyz_geq@int_slack@2=0.0, status=SUCCESS


In [19]:
result_qubo = cplex_optimizer.solve(qubo)
print(result_qubo)

fval=8.0, x=1.0, y=1.0, z@0=1.0, z@1=1.0, z@2=0.0, xyz_leq@int_slack@0=0.0, xyz_leq@int_slack@1=0.0, xyz_leq@int_slack@2=0.0, xyz_geq@int_slack@0=0.0, xyz_geq@int_slack@1=1.0, xyz_geq@int_slack@2=0.0, status=SUCCESS


In [20]:
print("interpreting values of result_eq_bin:", lineq2penalty.interpret(result_qubo.x))
print("values of result_eq_bin:", result_eq_bin.x)

interpreting values of result_eq_bin: [1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
values of result_eq_bin: [1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.]


We can see that the result of the `interpret` method implies that both the original and converted problems have exactly the same solution. This is expected because the converted problem has exactly the same variables as the original problem, the objective has been modified in such a way that we do not have the constraints anymore in the converted problem.

Finally, let us see how we interpret the result of QUBO back to the solution of the original problem `qp`. The following code shows that the interpreted values are equivalent to the result of the original problem `qp`.

In [21]:
print("result_orig.x", result_orig.x)

x = result_qubo.x
for conv in [lineq2penalty, int2bin, ineq2eq]:
    x = conv.interpret(x)
print("interpreting result_qubo.x", x)

result_orig.x [1. 1. 3.]
interpreting result_qubo.x [1. 1. 3.]


In [22]:
import tutorial_magics

%qiskit_version_table
%qiskit_copyright